In [3]:
import requests
import json
import numpy as np
from sklearn import linear_model
from datetime import datetime

import time

#  CoinMarketCap API 
api_key = "2d97db46-38e6-4a6d-9240-2b2704e8e07b"

#  symbols ETH , BTC 
symbols = ["ETH", "BTC"]

#  URL
url = "https://pro-api.coinmarketcap.com/v1/cryptocurrency/quotes/latest"

#  query parameters
params = {
    "symbol": ",".join(symbols),
    "convert": "USDT"
}

#  headers
headers = {
    "Accepts": "application/json",
    "X-CMC_PRO_API_KEY": api_key
}


'''for mimimum delay there are  alot of methods ,caching, load balacning , mimimze external dependencies and using websocket ,
but since i need to extarct the data once per minute in real time and since i am not using  binance(Blocked in Russia), so for this applciation , we just
can use one  request per mintue ( third-party APIs, can help reduce the overall response time of the API.)  with asseration handling using Try-exept..etc .
'''
print("In Task the required time's inrerval is 60 mintues ")
interval=input("Enter the time's interval in minutes  that you want the analysis to be into ")
interval=int(interval)
#  current date and time
now = datetime.now()  
changes_data = []
# print(now)
max_change = 0.0



def correlation(btc_array, eth_array):
    btc = np.array(btc_array).reshape((-1, 1))
    eth = np.array(eth_array)
    model_btc = linear_model.LinearRegression()
    model_btc.fit(btc, eth)
    eth_by_btc_predicted=model_btc.predict(btc)
    eth_residuals = eth - eth_by_btc_predicted

    model_eth = linear_model.LinearRegression()
    model_eth.fit(eth_residuals.reshape((-1, 1)), eth)
    prices = model_eth.predict(eth_residuals.reshape((-1, 1)))
    changes = eth - prices
    return changes, prices



def fun_analysis(BTC_data,ETH_data):
    global max_change 
    changes, prices = correlation(BTC_data, ETH_data)
    changes = np.array(changes)
    prices = np.array(prices)
    ind = np.argmax(changes)
    # print(cajnge)
    # print(abs(changes[ind]),abs(prices[ind]),y[ind])
    change_current = abs(changes[ind] / prices[ind]) * 100
    # change_current = abs(changes[0] / prices[0]) * 100
    changes_data.append(change_current)
    slice_value = int(-interval)  
    changes_data_per_hour = changes_data[slice_value:]
    max_changes_per_hour = float(max(changes_data_per_hour))
    print('max_changes_per_hour',max_changes_per_hour)

    if max_change != max_changes_per_hour and max_changes_per_hour > 0.01:
        max_change = max_changes_per_hour
        print(f'Price of ETHUSDT has changed by {max_change:.4f}% in the last {interval} minutes.')

run_minutes=now.minute

def get_price_data():
    calling_fun_time=time.time()
    global BTC_data, ETH_data, run_minutes
    BTC_data = []
    ETH_data = []
    while True:
      try:
          now = datetime.now() 
          # now = datetime.now()  
          # minutes = now.minute

          if (run_minutes - now.minute) !=0:
              print("at minute:",run_minutes)
              run_minutes=now.minute
              formatted_time = now.strftime("%Y-%m-%d %H:%M")
              try:
                  url = "https://pro-api.coinmarketcap.com/v1/cryptocurrency/quotes/latest"
                  # Send the request and get the response
                  response = requests.get(url, headers=headers, params=params)

                    # Check if the request was successful
                  if response.status_code == 200:
                        data = response.json()
                        for symbol in symbols:
                            price = data["data"][symbol]["quote"]["USDT"]["price"]
                            print(f"The current price of {symbol}/USDT  at {formatted_time} is {price:.6f} USD")
                            if symbol == "BTC":
                                BTC_data.append(price)
                            elif symbol == "ETH":
                                ETH_data.append(price) 

                  print( len(ETH_data),len(BTC_data) )
                  if ( ( len(ETH_data) % interval==0) & (len(ETH_data) !=0 )) :
                      fun_analysis(BTC_data[-interval:],ETH_data[-interval:])
              except:
                    pass
      except KeyboardInterrupt:
          print("Stopping loop...")
          break
                        
#           running_time=time.time()
#           # if (running_time-calling_fun_time)/3600)
            

       


get_price_data()
# btc_array=[]
# eth_array=[]
# import math
# for i in range(60):
#   btc_array.append(i)
#   eth_array.append(12*i)









In Task the required time's inrerval is 60 mintues 
Enter the time's interval in minutes  that you want the analysis to be into 3
at minute: 59
The current price of ETH/USDT  at 2023-05-05 11:00 is 1903.737850 USD
The current price of BTC/USDT  at 2023-05-05 11:00 is 29098.626182 USD
1 1
Stopping loop...
